In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
import os
from glob import glob
import timm

In [41]:
import segmentation_models_pytorch as smp
from pprint import pprint

In [23]:
import pylibjpeg
import pydicom
from tqdm import tqdm 
import albumentations
import nibabel as nb

In [24]:
import torch

In [33]:
path = "/mnt/d/rsna-2022-cervical-spine-fracture-detection"
train = pd.read_csv(f"{path}/train.csv")

train.shape

(2019, 9)

In [34]:
train_bb = pd.read_csv(f"{path}/train_bounding_boxes.csv")
train_bb.head()


,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
2,1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
3,1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
4,1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137


In [35]:

seg_paths = glob(f"{path}/segmentations/*")
seg = pd.DataFrame({'path': seg_paths})
seg['StudyInstanceUID'] = seg['path'].apply(lambda x:x.split('/')[-1][:-4])
seg = seg[['StudyInstanceUID', 'path']]
print(seg.shape)

(87, 2)


In [36]:
def get_observation_data(path):
    
    dataset = pydicom.read_file(path)
    meta_data = {
        "Rows" : dataset.get("Rows"),
        "Columns" : dataset.get("Columns"),        
        "SOPInstanceUID" : dataset.get("SOPInstanceUID"),
        "ContentDate" : dataset.get("ContentDate"),
        "SliceThickness" : dataset.get("SliceThickness"),
        "InstanceNumber" : dataset.get("InstanceNumber"),
        "ImagePositionPatient" : dataset.get("ImagePositionPatient"),
        "ImageOrientationPatient" : dataset.get("ImageOrientationPatient"),
    }
    str_columns = ["SOPInstanceUID", "ContentDate", 
                   "SliceThickness", "InstanceNumber"]
    for k in str_columns:
        meta_data[k] = str(dataset.get(k)) if k in dataset else None
        
        
    return meta_data

In [42]:
ex_path = f"{path}/train_images/1.2.826.0.1.3680043.10001/109.dcm"
example = get_observation_data(ex_path)
pprint(example)

{'Columns': 512,
 'ContentDate': '20220727',
 'ImageOrientationPatient': [1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000],
 'ImagePositionPatient': [-52.308, -27.712, -60.218],
 'InstanceNumber': '109',
 'Rows': 512,
 'SOPInstanceUID': '1.2.826.0.1.3680043.10001.1.109',
 'SliceThickness': '0.625000'}


In [43]:
dicom_train = []
for file in tqdm(range(len(train))):
    
    dt = train.iloc[file, :]
    
    dcm = glob(f"{path}/train_images/{dt.StudyInstanceUID}/*")
    for dcm_path in dcm:
        try:
            meta_data = get_observation_data(dcm_path)
            dicom_train.append(meta_data)
        except:
            continue
    # Convert into df
    meta_train_data = pd.DataFrame(data=dicom_train, columns=example.keys())
    # Export information
    meta_train_data.to_csv("meta_train.csv", index=False)
            


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2019/2019 [3:27:55<00:00,  6.18s/it]


In [47]:
#Create csv of train.csv for segmentation data so you have meta data on those files 
(meta_train_data['SOPInstanceUID'].nunique())

711104

https://www.kaggle.com/code/raddar/convert-dicom-to-np-array-the-correct-way

